In [1]:
import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" # or any {‘0’, ‘1’, ‘2’}
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
import tensorflow_probability as tfp
#from tf_keras import layers, Model, models
from tensorflow.keras import layers, Model, models, metrics, optimizers
#from tensorflow import keras
import keras
import tensorflow_datasets as tfds2
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import itertools
import gc
import os
import json
import inspect
import shutil
# tf.random.set_seed(1)

2024-12-11 13:52:39.713971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733921559.735120   74811 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733921559.741438   74811 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 13:52:39.764132: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.18.0
Num GPUs Available:  1


/home/max/miniconda3/envs/tf-wsl2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data loading + processing

### Geno Data

In [2]:
child_pheno_data = pd.read_csv("./data/pheno_data/pheno_tassel_dist.csv", 
                               index_col=["pop", "individual"])
parental_geno_data = pd.read_csv("./data/parent_genos_ordered.csv")
child_geno_data = pd.read_csv("./data/all_child_genos_ordered.csv",
                               index_col=["pop", "individual"])
child_pheno_data = pd.read_csv("./data/pheno_data/pheno_tassel_dist.csv", 
                               index_col=["pop", "individual"])
child_geno_data = child_geno_data.reindex(child_pheno_data.index).reset_index()
parent_pheno_data = pd.read_csv("./data/pheno_data/parents_tassel.csv")


# Get population per sample, split populations into oop and train
pop_per_sample = child_geno_data["pop"].to_numpy()
test_pops = np.random.choice(np.unique(pop_per_sample), 6, replace = False)
train_eval_pops_idx = np.where(~np.isin(pop_per_sample, test_pops))[0]
eval_idx = np.random.choice(train_eval_pops_idx, 1000, replace = False)
train_idx = train_eval_pops_idx[~np.isin(train_eval_pops_idx, eval_idx)]
test_idx = np.where(np.isin(pop_per_sample, test_pops))[0]
pop_per_sample_train_eval = pop_per_sample[train_eval_pops_idx]
pop_per_sample_test = np.delete(pop_per_sample, train_eval_pops_idx)

assert ~np.any(np.isin(eval_idx, train_idx))
assert ~np.any(np.isin(eval_idx, test_idx))
assert ~np.any(np.isin(test_idx, train_idx))

In [3]:
all_geno_combs = list(itertools.product(["A", "T", "G", "C"], ["A", "T", "G", "C"])) + [("-", "-")]
all_geno_combs = np.unique(["".join(np.sort(cur_genos)) for cur_genos in all_geno_combs])

In [4]:
def split_data(data, train_idx = train_idx, eval_idx = eval_idx, test_idx = test_idx):
    return data[train_idx, ...], data[eval_idx, ...], data[test_idx, ...]

In [5]:
p1_ids = child_geno_data['p1_id'].to_numpy()
p2_ids = child_geno_data['p2_id'].to_numpy()

# Parent 1 is technically always maize B73
# -> shuffle parent ids so model isnt fed same p1 input across all samples
p_genos_ids = np.stack([p1_ids, p2_ids], axis = 1)
[np.random.shuffle(cur_sub) for cur_sub in p_genos_ids]

## Extract parent phenotypes
def extract_phenos(ids, parent_data, extract_col):
    return np.array([parent_data.loc[parent_data["p_id"] == cur_id, extract_col].to_numpy().flatten()
                     for cur_id in ids])

## Extract parental phenos from pandas df
p1_phenos = extract_phenos(p_genos_ids[:, 0], parent_pheno_data, ["mean_value", "sd_value"])
p2_phenos = extract_phenos(p_genos_ids[:, 1], parent_pheno_data, ["mean_value", "sd_value"])
child_pheno_np = child_pheno_data[["val_mean", "val_sd"]].to_numpy()

## Stack parental phenotypes, split to oop and train + eval
parents_phenos = np.stack([p1_phenos, p2_phenos], axis=1)
parents_phenos_np_train, parents_phenos_np_eval, parents_phenos_np_test = split_data(parents_phenos)
child_pheno_np_train, child_pheno_np_eval, child_pheno_np_test = split_data(child_pheno_np)

## Extract parental genos, split to oop and train + eval
parental_geno_red = parental_geno_data.iloc[:, 1:]
p1_genos_np = extract_phenos(p_genos_ids[:, 0], parental_geno_red, parental_geno_red.columns[:-1])
p2_genos_np = extract_phenos(p_genos_ids[:, 1], parental_geno_red, parental_geno_red.columns[:-1])
p_genos_np = np.stack([p1_genos_np, p2_genos_np], axis = 1)
p_genos_np_train, p_genos_np_eval, p_genos_np_test = split_data(p_genos_np)

# Split child geno types into train + (in population) evaluation error and OOP
# IP data == eval, OOP == test
child_geno_np_train, child_geno_np_eval, child_geno_np_test = split_data(child_geno_data.iloc[:, 2:-2].to_numpy())
child_geno_np = child_geno_data.iloc[:, 2:-2].to_numpy()

In [6]:
# Construct np arrays from individual arrays, split train+eval array into individual
# train and eval arrays
# Technically this could be achieved way simpler by splitting a tf dataset, but
# since the genotype arrays dont contain the population id this is the only way i found
# to track them

# pop_per_sample_train = np.delete(pop_per_sample_train_eval, eval_samples)
# pop_per_sample_eval = pop_per_sample_train_eval[eval_samples]

In [7]:
# Convert data into one hot encodings and stack to tensors

def one_hot_encode(data, vocab = all_geno_combs):
    encoded_list = [np.float16(data == cur_class) for cur_class in vocab]
    return tf.stack(encoded_list, axis = -1)


child_geno_tensor_train = one_hot_encode(child_geno_np_train)
p_genos_tensor_train = one_hot_encode(p_genos_np_train)

child_geno_tensor_eval = one_hot_encode(child_geno_np_eval)
p_genos_tensor_eval = one_hot_encode(p_genos_np_eval)

child_geno_tensor_test = one_hot_encode(child_geno_np_test)
p_genos_tensor_test = one_hot_encode(p_genos_np_test)

child_geno_tensor = one_hot_encode(child_geno_np)
p_genos_tensor = one_hot_encode(p_genos_np)

I0000 00:00:1733921581.481737   74811 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1733921581.567587   74811 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4699 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1


In [8]:
# train_dataset -> actual training
# eval -> test dataset without seeing children, shared parents with train_dataset (IP)
# test_dataset -> test dataset without seeing children + parents (OOP)

geno_train_dataset = tf.data.Dataset.from_tensor_slices((p_genos_tensor_train, child_geno_tensor_train))
pheno_train_dataset = tf.data.Dataset.from_tensor_slices((parents_phenos_np_train, child_pheno_np_train))
train_dataset = tf.data.Dataset.zip((geno_train_dataset, pheno_train_dataset))
train_dataset = train_dataset.shuffle(int(1e6))
train_dataset = train_dataset.batch(256, drop_remainder=True)

geno_test_dataset = tf.data.Dataset.from_tensor_slices((p_genos_tensor_test, child_geno_tensor_test))
pheno_test_dataset = tf.data.Dataset.from_tensor_slices((parents_phenos_np_test, child_pheno_np_test))
test_dataset = tf.data.Dataset.zip((geno_test_dataset, pheno_test_dataset))
test_dataset = test_dataset.shuffle(buffer_size=int(1e6))
test_dataset = test_dataset.batch(p_genos_tensor_test.shape[0], drop_remainder=True)

## Model definition

In [12]:
# Moved defintions to separate python scripts
# inspect_dir = "./data/reg_vae/lim_kl_scale_01_reg_w_closs_smooth/"
files_to_load = ["reg_vae_helpers.py", "cur_encoder.py", "cur_decoder.py",
    "reg_autoencoder.py", "geno_to_trait_model.py", "regression_vae.py"]
for cur_file in files_to_load:
    execfile(cur_file)
cur_base_dir = "./data/reg_vae/lim_kl_scale_01_01_reg_w_closs_smooth_02/"
if not os.path.isdir(cur_base_dir):
    os.mkdir(cur_base_dir)

## Grid testing
* Deprecated, need to rerun once satisfying VAE structure found
* Test hyperparamters in grid like fashion

In [10]:

def fit_per_para(encoder_w, decoder_d, epochs = 100):
    print("###########################################")
    print(f"current encoder width: {encoder_w}")
    print(f"current decoder depth: {decoder_d}")
    true_classes = np.argmax(geno_tensor, axis = 1).reshape((-1))

    # used for focal loss
    inv_class_frequencies = 1 - ((np.unique(true_classes, return_counts=True)[1] / len(true_classes)))

    focal_loss = tf.keras.losses.CategoricalCrossentropy(axis = 1,
                                                          from_logits=False)

    focal_optimizer = optimizers.Lion(learning_rate=1e-4)
    model = autoencoder(100, encoder_width=encoder_w, decoder_depth=decoder_d)
    model.compile(optimizer=focal_optimizer, loss = focal_loss, run_eagerly=False,
                      metrics=[metrics.CategoricalAccuracy()])
    model_train_loss = model.fit(train_dataset, epochs=epochs, shuffle=True,
                                 validation_data = test_dataset)
    true_classes = np.argmax(test_labels, axis = 1).reshape((-1))
    pred_classes = np.argmax(model(test_labels), axis = 1).reshape((-1))
    class_acc = [np.sum((pred_classes == true_classes) & (pred_classes == cur_class))/
           np.sum(true_classes == cur_class) for cur_class in np.unique(true_classes)]
    print(class_acc)
    print(pd.crosstab(pd.Series(pred_classes, name="Predicted"), pd.Series(true_classes, name="Actual")))
    out = [class_acc, model_train_loss]
    del model
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()

    return out

In [ ]:
paras = list(itertools.product(list(range(5,0,-1)),list(range(5,0, -1))))

In [ ]:
res_per_para = [fit_per_para(cur_paras[0], cur_paras[1], epochs = 100) for cur_paras in paras]

###########################################
encoder_w: 5
decoder_d: 5
Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 144ms/step - categorical_accuracy: 0.0014 - loss: 1.1271 - val_categorical_accuracy: 0.0020 - val_loss: 1.0366
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - categorical_accuracy: 0.0020 - loss: 1.0030 - val_categorical_accuracy: 6.5104e-04 - val_loss: 0.9019
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.0025 - loss: 0.8578 - val_categorical_accuracy: 0.0290 - val_loss: 0.7626
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - categorical_accuracy: 0.0299 - loss: 0.7604 - val_categorical_accuracy: 0.0280 - val_loss: 0.7459
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - categorical_accuracy: 0.0287 - loss: 0.7447 - val_categorical_accuracy: 0.0238 - val_loss: 0.7372
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - categorical_accuracy: 0.0256 - loss: 0.7354 - val_categorical_accuracy: 0.0280 - val_loss: 0.7304
Epoch 7/200


In [ ]:
res_dict = {str(paras[cur_para_id]):[res_per_para[cur_para_id][0], res_per_para[cur_para_id][1].history] for cur_para_id in range(len(paras))}
res_out = open(cur_base_dir + "grid_test.txt", "w")
res_out.write(json.dumps(res_dict))
res_out.close()

In [ ]:
res_deep_encoder_thining = json.loads(open(cur_base_dir + "grid_test.txt").read())
dt_keys = list(res_deep_encoder_thining.keys())
dt_acc = [res_deep_encoder_thining[cur_dict][0] for cur_dict in dt_keys]
dt_train = [res_deep_encoder_thining[cur_dict][1]["loss"] for cur_dict in dt_keys]
dt_val = [res_deep_encoder_thining[cur_dict][1]["val_loss"] for cur_dict in dt_keys]

In [ ]:
dt_acc_df = pd.DataFrame(np.array(dt_acc))
dt_acc_df["fit"] = dt_keys

dt_train_df = pd.DataFrame(np.array(dt_train))
dt_train_df["fit"] = dt_keys

dt_train_df["loss_type"] = "train"
dt_val_df = pd.DataFrame(np.array(dt_val))
dt_val_df["fit"] = dt_keys
dt_val_df["loss_type"] = "test"
dt_train_df = pd.concat((dt_train_df, dt_val_df))

In [ ]:
dt_train_df.to_csv(cur_base_dir + "model_5_train.csv", sep=",")
dt_acc_df.to_csv(cur_base_dir + "model_5_acc.csv", sep=",")

## Individual Models

In [13]:
# reg_vae_optimizer = optimizers.Adam(learning_rate=1e-3)#, use_ema = True, gradient_accumulation_steps = 3)# ema_overwrite_frequency = int(1e3))
reg_vae_model = reg_vae(100, 5, 5)
reg_vae_model.compile(run_eagerly = False)

In [ ]:
res = train_and_get_results(reg_vae_model, epochs = 150,
    base_dir = cur_base_dir, files_to_backup = files_to_load)


Epoch 1/150


/home/max/miniconda3/envs/tf-wsl2/lib/python3.9/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/max/miniconda3/envs/tf-wsl2/lib/python3.9/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'decoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


## VAE model analysis

In [18]:
encoder_model = reg_vae_model.encoder
decoder_model = reg_vae_model.decoder
train_mean, train_logvar = encoder_model(tf.concat([p_genos_tensor_train, tf.expand_dims(child_geno_tensor_train, 1)], axis = 1))
train_embedding = reg_vae_model.sample_z(train_mean, train_logvar)
train_pred = decoder_model(p_genos_tensor_train, train_embedding)
train_pred = np.argmax(train_pred, axis = -1)
train_pred_df = pd.DataFrame(train_pred)
train_pred_df["d_type"] = "pred"
truth_df = pd.DataFrame(np.argmax(child_geno_tensor_train[:, 0, :], axis = -1))
truth_df["d_type"] = "truth"
full_train_df = pd.concat([train_pred_df, truth_df], axis = 0)
full_train_df.to_csv(cur_base_dir + "train_pop_pred.csv")

mean_embed_df = pd.DataFrame(train_mean.numpy())
mean_embed_df["individual"] = child_geno_data.loc[train_idx, "individual"].to_numpy()
mean_embed_df["pop"] = child_geno_data.loc[train_idx, "pop"].to_numpy()
mean_embed_df["p1_id"] = child_geno_data.loc[train_idx, "p1_id"].to_numpy()
mean_embed_df["p2_id"] = child_geno_data.loc[train_idx, "p2_id"].to_numpy()
mean_embed_df.to_csv(cur_base_dir + "train_pop_mean_embed.csv")

logvar_embed_df = pd.DataFrame(train_logvar.numpy())
logvar_embed_df["individual"] = child_geno_data.loc[train_idx, "individual"].to_numpy()
logvar_embed_df["pop"] = child_geno_data.loc[train_idx, "pop"].to_numpy()
logvar_embed_df["p1_id"] = child_geno_data.loc[train_idx, "p1_id"].to_numpy()
logvar_embed_df["p2_id"] = child_geno_data.loc[train_idx, "p2_id"].to_numpy()
logvar_embed_df.to_csv(cur_base_dir + "train_pop_logvar_embed.csv")

In [19]:
test_mean, test_logvar = encoder_model(tf.concat([p_genos_tensor_test, tf.expand_dims(child_geno_tensor_test, 1)], axis = 1))
test_embedding = reg_vae_model.sample_z(test_mean, test_logvar)
test_pred = decoder_model(p_genos_tensor_test, test_embedding)
test_pred = np.argmax(test_pred, axis = -1)
test_pred_df = pd.DataFrame(test_pred)
test_pred_df["d_type"] = "pred"
truth_df = pd.DataFrame(np.argmax(child_geno_tensor_test[:, 0, :], axis = -1))
truth_df["d_type"] = "truth"
full_test_df = pd.concat([test_pred_df, truth_df], axis = 0)
full_test_df.to_csv(cur_base_dir + "new_pop_pred.csv")

mean_embed_df = pd.DataFrame(train_mean.numpy())
mean_embed_df["individual"] = child_geno_data.loc[train_idx, "individual"].to_numpy()
mean_embed_df["pop"] = child_geno_data.loc[train_idx, "pop"].to_numpy()
mean_embed_df["p1_id"] = child_geno_data.loc[train_idx, "p1_id"].to_numpy()
mean_embed_df["p2_id"] = child_geno_data.loc[train_idx, "p2_id"].to_numpy()
mean_embed_df.to_csv(cur_base_dir + "train_pop_mean_embed.csv")

logvar_embed_df = pd.DataFrame(train_logvar.numpy())
logvar_embed_df["individual"] = child_geno_data.loc[train_idx, "individual"].to_numpy()
logvar_embed_df["pop"] = child_geno_data.loc[train_idx, "pop"].to_numpy()
logvar_embed_df["p1_id"] = child_geno_data.loc[train_idx, "p1_id"].to_numpy()
logvar_embed_df["p2_id"] = child_geno_data.loc[train_idx, "p2_id"].to_numpy()
logvar_embed_df.to_csv(cur_base_dir + "train_pop_logvar_embed.csv")

In [22]:
# Load stored model
# Moved defintions to separate python scripts
files_to_load = ["reg_vae_helpers.py", "cur_encoder.py", "cur_decoder.py",
    "reg_autoencoder.py", "geno_to_trait_model.py", "regression_vae.py"]
for cur_file in files_to_load:
    execfile(cur_file)

loaded_model = keras.models.load_model(
    "./data/reg_vae/lim_kl_scale_001/model.keras",
    custom_objects = {"encoder" : encoder, "decoder" : decoder, "reg_vae" : autoencoder,
        "elbo_loss" : elbo_loss, "rec_loss_fn" : rec_loss_fn, "reg_loss_fn" : reg_loss_fn, "feature_drop_layer": feature_drop_layer}
)
cur_model = loaded_model

/home/max/miniconda3/envs/tf-wsl2/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 350 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/max/miniconda3/envs/tf-wsl2/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 646 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Prediction per population

In [15]:
geno_np_arr = np.stack(geno_np_layers, axis = -1)

In [ ]:
# Loop through populations in dataset and store genotype reconstruction accuracies
def class_acc_per_pop(model, data, sample_pops):
    pops = []
    class_accs_in_oop = {class_id: [] for class_id in range(data.shape[-1])}
    for cur_pop in np.unique(pop_per_sample):
        pop_idx = sample_pops == cur_pop
        pops.append(cur_pop)
        cur_geno_tensor = data[pop_idx, ...]
        pred = model(cur_geno_tensor)
        true_classes = tf.argmax(cur_geno_tensor[:, 0, ...], axis = -1)
        for class_id in range(pred.shape[-1]):

            class_pred = tf.equal(tf.argmax(pred, axis = -1), class_id)
            class_y = tf.equal(true_classes, class_id)
            correct_pred = tf.cast(tf.logical_and(class_pred, class_y), tf.int32)

            acc = tf.reduce_sum(correct_pred)/tf.reduce_sum(tf.cast(class_y, tf.int32))
            class_accs_in_oop[class_id].append(acc.numpy())    
    res = pd.DataFrame(class_accs_in_oop)
    res["pop"] = pops
    return res

In [ ]:
res_df = class_acc_per_pop(cur_model, geno_np_arr, pop_per_sample)
res_df["data_type"] = "train"
res_df.loc[np.isin(res_df["pop"], test_pops), "data_type"] = "oop"

In [ ]:
eval_df = class_acc_per_pop(cur_model, geno_tensor_eval.numpy(), eval_pops)
eval_df["data_type"] = "eval"

In [54]:
pd.concat([res_df, eval_df]).to_csv("./data/var_autoencoder/with_unseen_children_vanilla_vae/acc_per_pop.csv")